Spark-Session erstellen und notwendige Frameworks importieren

In [1]:
import findspark
findspark.init()
import pyspark

import pyspark.sql

sc = pyspark.SparkContext(appName="FlightData")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/29 20:03:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Einlesen der Datei mit allen weltweiten Flugplätzen und Filtern auf die Region "EU" sowie Airport Typ "medium_airport" und "large_airport"

(Hier stimmt was noch nicht, laut CSV müssten es eigentlich mehr als 976 Flugplätze sein!)

In [107]:
airportRDD = sc.textFile("/Users/jankozirkelbach/Library/CloudStorage/GoogleDrive-janko.zirkelbach@hotmail.com/Meine Ablage/Master/Advanced Data Engineering/Projekt/airports.csv")
header = airportRDD.first()

airportRDD = (
    airportRDD.filter(lambda line: line != header)
    .filter(lambda line: line.split(",")[7].strip('"') in ["EU"])
    .filter(lambda line: line.split(",")[2].strip('"') in ["medium_airport", "large_airport"])
)

Anzahl_Flughäfen_gesamt = airportRDD.count()

Anzahl_Flugplätze_kategorisiert = (
    airportRDD.map(lambda line: line.split(",")[2].strip('"'))
    .countByValue()
)

print(f"Die Anzahl der europäischen mittelgroßen und großen Flughäfen beträgt: {Anzahl_Flughäfen_gesamt}")
print(f"Davon mittelgroße Flughäfen: {Anzahl_Flugplätze_kategorisiert.get("medium_airport", 0)}")
print(f"Davon große Flughäfen: {Anzahl_Flugplätze_kategorisiert.get("large_airport", 0)}")

Die Anzahl der europäischen mittelgroßen und großen Flughäfen beträgt: 976
Davon mittelgroße Flughäfen: 864
Davon große Flughäfen: 112


Einlesen und filtern aller weltweit zugelassenen Flugobjekte, sortiert nach den Airlines mit den meisten Flugzeugen

In [108]:
aircraftRDD = sc.textFile("/Users/jankozirkelbach/Library/CloudStorage/GoogleDrive-janko.zirkelbach@hotmail.com/Meine Ablage/Master/Advanced Data Engineering/Projekt/aircraft-database-complete-2024-10.csv")
header = aircraftRDD.first()

Anzahl_registrierte_Flugobjekte = aircraftRDD.count()
print(f"Anzahl der weltweit gemeldeten, eindeutigen Flug- und Bodenobjekte {Anzahl_registrierte_Flugobjekte}")

aircraftRDD = (
    aircraftRDD.filter(lambda line: line != header)
    .filter(lambda line: len(line.split(","))>16)
    .map(lambda line: line.split(",")[17].strip('"'))
    .filter(lambda airline: airline != "''")
    .map(lambda airline: (airline, 1))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: x[1], ascending=False)            
)

top_20_airlines = aircraftRDD.take(20)

print("Top 20 Airlines nach Anzahl der registrierten Flugzeuge:")
for airline, count in top_20_airlines:
    print(f"{airline}: {count}")

Anzahl der weltweit gemeldeten, eindeutigen Flug- und Bodenobjekte 601338
Top 20 Airlines nach Anzahl der registrierten Flugzeuge:
United States Air Force: 2223
Delta Air Lines: 781
American Airlines: 657
Corporate Airlink: 649
Federal Express: 560
Royal Air Force: 503
United Airlines: 451
Southwest Airlines: 280
German Air Force: 277
Boeing: 274
United Parcel Service: 270
Indian Air Force: 236
Force Aerienne Francaise: 223
Gulfstream Aerospace: 201
Bombardier: 190
Ryanair: 159
Qatar Airways: 157
Skywest Airlines: 153
Aeroflot Russian Airlines: 147
Royal Netherlands Air Force: 144


Einlesen und filtern der Flugzeug Bewegungsdaten nach Landungen mit darauffolgendem Start

In [ ]:
flightdata_file = "/Users/jankozirkelbach/Library/CloudStorage/GoogleDrive-janko.zirkelbach@hotmail.com/Meine Ablage/Master/Advanced Data Engineering/Projekt/20230126_merged.csv"
